# Order algorithm

In [11]:
import numpy as np
def Stange_method(n,g,b=10,c=10,keepFT=False):    
    # n = composite modulus
    # g = base g<n
    # b = size of factor base
    # c = number of additional relations
    # OUTPUT:  order of g mod n
    
    #-------------------phase 1: Relation finding------------------------- 
    relsFound = 0               # number of relations found
    relsDesired=b+c   # number of relations desired
    rand_x=[]                          
    FT=matrix(b,b+c)   # relation Matrix           
    while( relsFound < relsDesired ):
        
        x=randint(2,n-1)
        while x in rand_x:
            x=randint(1,n-1)
        
        residue = ZZ(Mod(g,n)^x)
        f=[]

        #-------------------Attempt factorisation of g^x------------------------------
        for p in primes_first_n(b):
            exponent=0
            while Mod(residue,p)==0:
                residue=residue/p           
                exponent+=1
            f.append(exponent)
        #------------------------------Append the vector if the relationship is found---------------------
        if residue==1:
#             print(g,"^{",x,"}&=",latex(factor(ZZ(Mod(g,n)^x))),",\\\\")
            f=vector(f)
            rand_x.append(x)
            FT[:,relsFound]=f
            relsFound+=1
    #------------------------------phase 2: Linear Algebra---------------------------------------
#     print(latex(FT))
    kernel_basis=FT.right_kernel().basis()[:c] # obtain the basis
#     print(latex(matrix(kernel_basis)))
    
        #-----------------------Find alphas--------------------------------
    alpha=[]
    for base_vec in kernel_basis:
        alp=np.dot(base_vec,rand_x) 
        alpha.append(alp)  #list of all the alphas
        
        
    #----------------------phase 3: Computing order of g mod n----------------------------
#     print(latex(alpha))
    order=Integer(gcd(alpha)) # probabilistic 
    
    if Mod(g^order,n)!=1:
        print("Error: the order is incorrect")
        return "Error"
    if keepFT:
        return order,FT,rand_x
    return order


# Even Order Method

In [12]:
def EO_method(n,g,order):
    order/=2
    M=Mod(g^order,n)
    if M!=1 and M!=-1:
        return gcd(g^order-1,n) , n/gcd(g^order-1,n) 
    elif M==1 and order%2==0:
        return EO_method(n,g,order)
    else: return False
    

n=9983
g=8
print(Stange_method(n,g))
order=Mod(g,n).multiplicative_order()
trtr=Mod(g^(order/2),n)-1
fact1=gcd(trtr,n)
print(fact1)
print(n/67, 67*149)

1628
67
149 9983


In [13]:
def EO_factor_method(n,b=10,c=10,keepFT=False):
    
    for g in range(2,n):
        print(g)
        if gcd(g,n)!=1:
            return gcd(g,n),n/gcd(g,n)
        order=Stange_method(n,g,b,c,keepFT)
        if order%2==0:

#             order/=2
#             M=Mod(g^order,n)
#             if M!=1 and M!=-1:
#                 return gcd(g^order-1,n) , n/gcd(g^order-1,n)
            
            factors=EO_method(n,g,order)
            if factors:
                return factors
    
        
        
        
#     n=4987*4993
n=105
f1,f2=EO_factor_method(n,b=10,c=10)
print(f1,f2,f1*f2==n)

2
21 5 True


# Ereka method

In [14]:
def eta(q,B):
    i=1
    while q^i<=B:
        i+=1
    return i-1


def E_factor_method(n,b=10,c=10,s=1,keepFT=False):
    g=randint(2,n-1)
    if gcd(g,n)!=1:
        return gcd(g,n),n/gcd(g,n)
    order=Stange_method(n,g,b,c,keepFT)
    
    m=s*n.nbits()
    prod=1
    for p in prime_range(m):
        q=eta(p,m)
        prod*=p^q
    r=order*prod
    twopower=(int(r.binary(),2) & (~(int( (r- 1).binary() ,2)) ) )
    o=r/twopower
    x=1
    while True:
#         x=randint(2,n-1)
        x+=1
        print(x)
        if gcd(x,n)!=1:
            return gcd(x,n),n/gcd(x,n)
        b=ZZ(Mod(x,n)^o)
        d=gcd(b-1,n)
        if d!=1:
            return d,n/d

        for i in range(twopower):
            b=ZZ(Mod(b,n)^2)
            d=gcd(b-1,n)
            if d!=1:
                return d,n/d
        
    

    
fac1,fac2=E_factor_method(101*11)
print(fac1,fac2,fac1*fac2)

2
11 101 1111


In [15]:
def E_factor_method(n,s=1,keepFT=False):
    order=4884    
    m=s*n.nbits()
    prod=1
    for p in prime_range(m):
        q=eta(p,m)
        prod*=p^q
    r=order*prod
    twopower=(int(r.binary(),2) & (~(int( (r- 1).binary() ,2)) ) )
    o=r/twopower
    x=1
    while True:
#         x=randint(2,n-1)
        x+=2
        if gcd(x,n)!=1:
            return gcd(x,n),n/gcd(x,n)
        b=ZZ(Mod(x,n)^o)
        print(b)
        d=gcd(b-1,n)
        print(d)
        if d!=1:
            return d,n/d

        for i in range(twopower):
            b=ZZ(Mod(b,n)^2)
            d=gcd(b-1,n)

            if d!=1:
                return d,n/d
        
    

    
fac1,fac2=E_factor_method(9983)
print(fac1,fac2,fac1*fac2)

9580
1
67 149 9983


# Single Kernel Method

In [16]:
import numpy as np
def SK_method(n,g,b=10):    
    # n = composite modulus
    # g = base g<n
    # b = size of factor base
    # OUTPUT:  factors of n
    
    #-------------------phase 1: Relation finding------------------------- 
    rank=0
    rand_x=[]                          
    F=[]   # relation Matrix
    
    while( rank==0 ):
        x=randint(1,n-1)
        while x in rand_x:
            x=randint(1,n-1)
        
        residue = ZZ(Mod(g,n)^x)
        f=[]

        #-------------------Attempt factorisation of g^x------------------------------
        for p in primes_first_n(b):
            exponent=0
            while Mod(residue,p)==0:
                residue=residue/p           
                exponent+=1
            f.append(exponent)

        #------------------------------Append the vector if the relationship is found---------------------
        if residue==1:
 #             print(g,"^{",x,"}&=",latex(factor(ZZ(Mod(g,n)^x))),",\\\\")
            f=vector(f)
            rand_x.append(x)
            F.append(f)
            #--------------- Periodically compute kernel----------------------------

            rank=matrix(F).transpose().right_kernel().rank()
    kernel_basis=matrix(F).transpose().right_kernel().basis()
    
    [order]=abs(np.dot(kernel_basis,rand_x))
    return ZZ(order)



n=1239*13*11
g=5
order=SK_method(n,g)

# this gives a multiple of the order


In [17]:
def SK_factor_method(n,b=10):    
    for g in range(2,n):
        print(g)
        if gcd(g,n)!=1:
            return gcd(g,n),n/gcd(g,n)

        order=SK_method(n,g,b)       
        
        if order%2==0:
            factors=EO_method(n,g,order)
            if factors:
                return factors
    
        
        
        
n=4987*4993
# n=550
f1,f2=SK_factor_method(n,b=10)
print(f1,f2,f1*f2==n)

2
4987 4993 True


# Discrete Logarithm, reuse matrix F

In [18]:
import numpy as np
#Index calculus

def IC(n,g,order,y,b=10,c=10):    
    # n = composite modulus
    # g = base g<n
    # b = size of factor base
    # c = number of additional relations
    # OUTPUT:  factors of n
    
    #-------------------phase 1: Relation finding------------------------- 
    relsFound = 0               # number of relations found
    relsDesired=b+c   # number of relations desired
    rand_x=[]                          
    FT=matrix(b,b+c)   # relation Matrix

    x=0
    while( relsFound < relsDesired ):
        
        x+=1
        
        residue = ZZ(Mod(g,n)^x)
        f=[]

        #-------------------Attempt factorisation of g^x------------------------------
        for p in primes_first_n(b):
            exponent=0
            while Mod(residue,p)==0:
                residue=residue/p           
                exponent+=1
            f.append(exponent)
        #------------------------------Append the vector if the relationship is found---------------------
        if residue==1:
#             print(g,"^{",x,"}&=",latex(factor(ZZ(Mod(g,n)^x))),",\\\\")
            f=vector(f)
            rand_x.append(x)
            FT[:,relsFound]=f
            relsFound+=1

    #------------------------------phase 2: Linear Algebra---------------------------------------
 # solve a=Fp

    F=FT.transpose()    
    Fring=F.change_ring(Integers(order))
    randring=vector(rand_x).change_ring(Integers(order))
    logp=Fring\randring     
    print(f"P={logp}")
    
        #-----------------------phase 3: Find one last relation--------------------------------
# find relatiob yg^k=g^xg^k
    k=-1
    residue=0
    while residue!=1:
        k+=1
        residue=Mod(y*(g^k),n)
        f=[]
        for p in primes_first_n(b):
            exponent=0
            while Mod(residue,p)==0:
                residue=residue/p           
                exponent+=1
            f.append(exponent)
    print(f,k)
    return np.dot(logp,f)-k
            

n=103
g=15
order=Mod(g,n).multiplicative_order()
print(f"the order is {order}")

y=15^18
power=IC(n,g,order,y,b=10,c=10)
print(f"the log computed is {power}")
print(Mod(g,n)^power==Mod(y,n))
# print(Mod(g,n)^power)

the order is 51
P=(47, 15, 37, 46, 13, 12, 39, 2, 21, 20)
[3, 2, 0, 0, 0, 0, 0, 0, 0, 0] 0
the log computed is 18
True


In [19]:
# def IC_factor_method(n,b=10,c=10):    
#     g=2
#     if gcd(g,n)!=1:
#         return gcd(g,n),n/gcd(g,n)
    
#     (order,FT,rand_x)=Stange_method(n,g,b,c,True) 
    
#     if order%2==0:
#         factors=EO_method(n,g,order)
#         if factors:
#             return factors
     
#     print(order)
#     F=FT.transpose()   
#     Fring=F.change_ring(Integers(order))
#     randring=vector(rand_x).change_ring(Integers(order))
#     logp=Fring\randring
#     print(logp)
#     for j in range(len(logp)):
#         print(Mod(2^logp[j],n))
        
        
#     # use logp to get new order
#     for i in range(3,4): #----_-_-_-----_---_- change this back
#         if gcd(i,n)!=1:
#             return gcd(i,n),n/gcd(i,n)
#         print(f"i is {i}")
#         #-------------Find one relation------------------------
        
#         k=-1
#         residue=0
#         while residue!=1:
#             k+=1
#             residue=Mod(i*(g^k),n)
#             print(residue)
#             f=[]
#             for p in primes_first_n(b):
#                 exponent=0
#                 while Mod(residue,p)==0:
#                     residue=residue/p           
#                     exponent+=1
#                 f.append(exponent)
#         print(f,residue)
#         log_g_i=np.dot(logp,f)-k
#         print(f"log_g(i)={log_g_i}")
#         print(f"the result is {g}^{log_g_i}={i}",Mod(g,n)^log_g_i,i)
#         #-------------------log_g(1)/log_g(i)=log_i(1)-----------------
#         order1=order/log_g_i
#         print(order1)
        
        
        
# n=4987*4993
# n=152942113

n=105
# f1,f2=
# IC_factor_method(n,b=10,c=10)
# print(f1,f2,f1*f2==n)

order=Mod(2,n).multiplicative_order()



the method above need that 2 generates 2,3,5,7 etc so that logp makes any sense.